##### Scrape school district info from the county assessor site.  This data is probably useful but this also helps demonstrate the ability to consume data from different sources

In [13]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random

In [4]:
data = pd.read_csv("AmesHousing.csv", "~")

In [5]:
_path = "https://beacon.schneidercorp.com/Application.aspx?AppID=165&LayerID=2145&PageTypeID=4&PageID=1108&KeyValue="

req_path = _path + "0526351010"

In [6]:
print req_path

https://beacon.schneidercorp.com/Application.aspx?AppID=165&LayerID=2145&PageTypeID=4&PageID=1108&KeyValue=0526351010


In [18]:
req = requests.get(req_path)

In [19]:
str(req)

'<Response [200]>'

In [17]:
# get html using selenium
# def strip_mine_path(path):
#     '''
#     Get the html for a parcel and try to pull out the school district
    
#     Return school district as string if successful, otherwise return 'error'
#     '''
# create 'browser'
driver = webdriver.Chrome()

# set size of window
driver.set_window_size(900, 800)

# request page
driver.get(req_path)

time.sleep(1)

# go path the accept page
driver.find_element_by_css_selector('a.btn.btn-default.button-1').click()


pids = data.PID
# pids = data.PID.iloc[212:]

# print pids

school_list = []

# format string into pid
# example PID: 05-22-160-070
# example data PID: 526301100
# converted: 05-26-301-100
for idx, pid in enumerate(pids):
    pid = str(pid)
    formatted = "{}-{}-{}-{}".format("0" + pid[0], pid[1:3], pid[3:6], pid[6:])

    
    
    # try to get the div that we want
    div = driver.find_elements_by_id("ctlBodyPane_ctl00_ctl01_lblSchoolDistrict")
    
#     print div
    
    # if the div is empty then there's no match, add a nan, otherwise add the 
    # correct item
    if div == []:
        school_list.append({"pid":pid, "school":np.nan})
        print "{:4d}: No match\r".format(idx+1)
    else:
        # school district
        school = div[0].get_attribute("innerHTML")

        school_list.append({"pid":pid, "school":school})
        print "{:4d}: School: {:<25}\r".format(idx+1, school)
    
    random_time = random.random() * 10.0
    time.sleep(random_time)
    
    # move to search tab
    driver.find_elements_by_class_name("tab-item-bar")[2].click()

    # search for next property
    _input = driver.find_element_by_id("ctlBodyPane_ctl02_ctl01_txtParcelID")
    _input.send_keys(formatted)
    _input.send_keys(Keys.ENTER)


time.sleep(2)

driver.close()

   1: School: AMES COMMUNITY SCHOOL    
   2: School: AMES COMMUNITY SCHOOL    
   3: School: AMES COMMUNITY SCHOOL    
   4: School: AMES COMMUNITY SCHOOL    
   5: School: AMES COMMUNITY SCHOOL    
   6: School: GILBERT COMMUNITY SCHOOL 
   7: School: GILBERT COMMUNITY SCHOOL 
   8: School: GILBERT COMMUNITY SCHOOL 
   9: School: GILBERT COMMUNITY SCHOOL 
  10: School: GILBERT COMMUNITY SCHOOL 
  11: School: GILBERT COMMUNITY SCHOOL 
  12: School: GILBERT COMMUNITY SCHOOL 
  13: School: GILBERT COMMUNITY SCHOOL 
  14: School: GILBERT COMMUNITY SCHOOL 
  15: School: GILBERT COMMUNITY SCHOOL 
  16: School: GILBERT COMMUNITY SCHOOL 
  17: School: GILBERT COMMUNITY SCHOOL 
  18: School: GILBERT COMMUNITY SCHOOL 
  19: School: GILBERT COMMUNITY SCHOOL 
  20: School: GILBERT COMMUNITY SCHOOL 
  21: School: AMES COMMUNITY SCHOOL    
  22: School: AMES COMMUNITY SCHOOL    
  23: School: AMES COMMUNITY SCHOOL    
  24: School: AMES COMMUNITY SCHOOL    
  25: School: AMES COMMUNITY SCHOOL    


In [61]:
print school_list

[{'school': u'AMES COMMUNITY SCHOOL', 'pid': '526301100'}, {'school': u'AMES COMMUNITY SCHOOL', 'pid': '526350040'}, {'school': u'AMES COMMUNITY SCHOOL', 'pid': '526351010'}, {'school': u'AMES COMMUNITY SCHOOL', 'pid': '526353030'}, {'school': u'AMES COMMUNITY SCHOOL', 'pid': '527105010'}, {'school': u'GILBERT COMMUNITY SCHOOL', 'pid': '527105030'}, {'school': u'GILBERT COMMUNITY SCHOOL', 'pid': '527127150'}, {'school': u'GILBERT COMMUNITY SCHOOL', 'pid': '527145080'}, {'school': u'GILBERT COMMUNITY SCHOOL', 'pid': '527146030'}, {'school': u'GILBERT COMMUNITY SCHOOL', 'pid': '527162130'}]


In [67]:
boink = pd.DataFrame(school_list)

In [68]:
boink.head(10)

,pid,school
0,526301100,AMES COMMUNITY SCHOOL
1,526350040,AMES COMMUNITY SCHOOL
2,526351010,AMES COMMUNITY SCHOOL
3,526353030,AMES COMMUNITY SCHOOL
4,527105010,AMES COMMUNITY SCHOOL
5,527105030,GILBERT COMMUNITY SCHOOL
6,527127150,GILBERT COMMUNITY SCHOOL
7,527145080,GILBERT COMMUNITY SCHOOL
8,527146030,GILBERT COMMUNITY SCHOOL
9,527162130,GILBERT COMMUNITY SCHOOL


In [74]:
len(boink)

156

In [70]:
school_back = school_list

In [75]:
len(school_list)

59

In [18]:
data = pd.DataFrame(school_list)

In [19]:
data.head()

,pid,school
0,526301100,AMES COMMUNITY SCHOOL
1,526350040,AMES COMMUNITY SCHOOL
2,526351010,AMES COMMUNITY SCHOOL
3,526353030,AMES COMMUNITY SCHOOL
4,527105010,AMES COMMUNITY SCHOOL


In [20]:
data.to_csv("school_data.csv")